# 线性模型

> $ f(\bf{x}) = \bf{\omega}^T\bf{x} + b $

In [1]:
# 生成数据集
# y = lambda^T * x + b
import numpy as np

lambdas = np.array([0.4, 0.6, 0.1])
b = 0.7

x = np.array(
    [[0.1, 0.3, 0.6],
    [0.5, 0.7, 0.1],
    [0.9, 1.1, 3.1],
    [0.1, 0.5, 0.3],
    [0.9, 0.1, 0.0],
    [0.0, 0.3, -0.3],
    [0.3, 0.5, 0.1],
    [-0.5, 0.6, 0.2]])

y = np.dot(x, lambdas)+ b+ (np.random.rand(8)-0.5)*0.001



## 最小二乘法

> $ (\omega^*, b^*) = argmin_{(\omega,b)}\sum_{i=1}^{m}(y_i-\omega x_i-b)^2 $

> $ \omega = \omega + \alpha * (\omega \sum_{i=1}^{m}x_i^2 - \sum_{i=1}^{m}(y_i-b)x_i) $

> $ b = b + \alpha * (mb - \sum_{i=1}^{m}(y_i-\omega x_i)) $

#### 向量形式

> $ \omega^* = argmin_{\omega}(y-X\omega)^T(y-X\omega) $

> $ \omega = \omega + X^T(X\omega - y) $

In [9]:
def least_square_method(x, y, a=0.01, eps=1e-9, maxstep=1000):
    X = np.concatenate([x, np.ones((x.shape[0],1))], axis=1)
    w = np.random.rand(X.shape[1])
    w_0 = np.zeros(X.shape[1])
    step = 0
    obj_0 = 0
    obj = np.sum( (np.dot(X, w) - y)**2 )
    while (np.abs(obj_0-obj) > eps and step < maxstep):
        w_0 = w.copy()
        obj_0 = obj
#         print (obj)
        w = w - a * (np.dot(X.T, np.dot(X, w)-y))
        obj = np.sum( (np.dot(X, w) - y)**2 )
        # print (np.sum((w_0-w)**2))
        if step%(maxstep//100) ==0:
            print(w)
        step += 1
    return w
        
least_square_method(x,y,maxstep=100000)

[ 0.42551761  0.47890737  0.20751749  0.18229606]
[ 0.40248517  0.61171114  0.09687605  0.69461181]
[ 0.40055201  0.60223152  0.0993752   0.69883716]


array([ 0.40037093,  0.60134342,  0.09960933,  0.69923301])

# 对数几率回归 logistic regression

> 不要轻易翻译为逻辑回归

> $ y = \frac{1}{1+e^{-z}} $

> 建立对数似然函数之后，求导得：

> $ \frac{\partial L}{\partial \theta} = \sum_{i=1}^{m} (y_i-\sigma(\theta^T x_i))x_i $

In [13]:
import pdb

def sigma(X):
    return 1/(1+np.e**(-X))
    
def logistic_regression(x, y, a=0.01, eps=1e-9, maxstep=1000, lambdas=0.8):
    dim = x.shape[1]
    theta = np.random.rand(dim)
    theta_0 = np.zeros(dim)
    obj = np.sum( (sigma(np.dot(x, theta)) -y)**2 ) + lambdas * np.sum(theta**2)
    obj_0 = 0
    step = 0
    while (np.abs(obj-obj_0) > eps) and step < maxstep:
        theta_0 = theta
        obj_0 = obj
        delta = np.dot((sigma(np.dot(x, theta)) - y) ,x) + lambdas * theta
        theta = theta - a * delta
        obj = np.sum( (sigma(np.dot(x, theta)) -y)**2 ) + lambdas * np.sum(theta**2)
        if step % (maxstep//100) == 0:
            print(obj, delta)
        step +=1
    print("Total step: %s" % step)
    return theta
        
y_logistic = (y > 1).astype('int')
logistic_regression(x, y_logistic, a=0.01, eps=1e-9, maxstep=10000)

2.76156376608 [-0.60982513  0.75410824  0.82503003]
2.11587137965 [-0.24958155  0.18703911  0.17256454]
2.17849185539 [-0.09462561  0.0500452   0.02909496]
2.22994792946 [-0.03338863  0.01594953  0.00595353]
2.25075640888 [-0.01150388  0.00551054  0.00148642]
2.25820632766 [-0.00393568  0.00195551  0.00041047]
2.26078317092 [-0.00134393  0.00069928  0.00011655]
2.26166498899 [ -4.58803038e-04   2.50480105e-04   3.27051605e-05]
2.26196578085 [ -1.56671834e-04   8.97110674e-05   8.82442859e-06]
2.26206830245 [ -5.35236374e-05   3.21110249e-05   2.21238070e-06]
2.26210324747 [ -1.82943300e-05   1.14854969e-05   4.78560953e-07]
2.26211516262 [ -6.25625531e-06   4.10520321e-06   6.72916748e-08]
2.26211922708 [ -2.14064868e-06   1.46630471e-06  -1.04439742e-08]
2.26212061419 [ -7.32847345e-07   5.23403751e-07  -1.51972342e-08]
2.26212108782 [ -2.51028107e-07   1.86719871e-07  -9.23945348e-09]
Total step: 1492


array([ 1.0278774 ,  0.24836494,  0.29922365])

In [17]:
# from sklearn
import sklearn.linear_model
lr = sklearn.linear_model.LogisticRegression(verbose=1, random_state=np.random.randint(1000000), warm_start=True)
lr.fit(x, y_logistic)
print (lr.coef_)
print (lr.intercept_)

[LibLinear][[ 0.87608388  0.21525248  0.28978609]]
[ 0.0376202]
